In [3]:
import numpy as np
from bpemb import BPEmb
from tensorflow.contrib import predictor

vocab_size = 3000
emb_dim = 100
bpemb_ru = BPEmb(lang='ru', vs=vocab_size , dim=emb_dim)
export_dir = 'export_dir/1555572340'
predict_fn = predictor.from_saved_model(export_dir)

def parse_fn(line, bpemb = bpemb_ru):
    '''
        функция кодировки строки:
        line- строка
    '''
    feature = np.array([bpemb.encode_ids(line)]).astype(np.int32)
    return feature, np.array([len(feature[0])])

def to_capital_latter(sentence):
    tmp = ''
    flag = True
    for c in sentence:
        if flag and c != ' ':
            tmp += c.upper()
            flag = False
        else:
            tmp += c
        if c in '.?!':
            flag = True
    return tmp

def predict(line, predict_fn=predict_fn):
    x, x_len = parse_fn(line)
    d = {1:2922, 2:2921, 3:2978, 4:2985, 5:2947, 6:2963}
    predict = predict_fn({'x':x, 'len':x_len })
    a = []
    for i in range(predict['lengths'][0]):
        a.append(predict['sequences'][0][i])
        if predict['prediction'][0][i] != 0:
            a.append(d[predict['prediction'][0][i]])
    return to_capital_latter(bpemb_ru.decode_ids(np.array(a)))


INFO:tensorflow:Restoring parameters from export_dir/1555572340/variables/variables


In [7]:
%%time
predict('Быть энтузиасткой сделалось ее общественным положением и иногда когда ей даже того не хотелось она чтобы не обмануть ожиданий людей знавших ее делалась энтузиасткой сдержанная улыбка игравшая постоянно на лице Анны Павловны хотя и не шла к ее отжившим чертам выражала как у избалованных детей постоянное сознание своего милого недостатка от которого она не хочет не может и не находит нужным исправляться')

CPU times: user 455 ms, sys: 550 ms, total: 1 s
Wall time: 177 ms


'Быть энтузиасткой сделалось ее общественным положением и иногда, когда ей даже того не хотелось она, чтобы не обмануть ожиданий людей знавших ее делалась энтузиасткой сдержанная улыбка игравшая постоянно на лице анны павловны, хотя и не шла к ее отжившим чертам выражала, как у избалованных детей постоянное со,знание своего милого недостатка, от которого она не хочет не может и не находит нужным исправляться.'